In [3]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

# Initial Prompt

- 0-shot

In [2]:
init_temp = """### Instruction ###
Act as a professional {role} working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Revise a ticket for compliance with these best practices and offer recommendations if needed.
The best practice to check is: 
{best_practice}

### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp, and content.
Components: Project components to which the issue belongs.
CreatedDate: The time and date the issue was created.
Creator: The person who created the issue.
Description: A detailed description of the issue.
IssueLinks: A list of links to related issues.
IssueType: The issue's purpose within the organization.
Labels: Labels to which this issue relates.
Priority: The issue importance in relation to other issues.
Project: The parent project to which the issue belongs.
Reporter: The person who found/reported the issue.
Resolution: A record of the issue's resolution, once resolved or closed.
ResolvedDate: The time and date the issue was resolved.
Status: The stage the issue is currently at in its lifecycle.
Summary: A brief one-line summary of the issue.
TimeSpent: Amount of time spent working on the issue.
VersionsAffected: The versions of the project affected by the issue.
VersionsFixed: Project versions in which the issue was (or will be) fixed.

You will receive this ticket from an issue tracker in JSON format.
Ticket: {ticket}
"""
init_prompt = PromptTemplate(template= init_temp, input_variables= ["role", "best_practice", "ticket"])

init_prompt.save("prompts/init/initPrompt_V1.1.2.json")

# Update Prompt (Best Practice)

- 0-shot
- ??

In [20]:
update_temp ="""Best Practice: Update Missing/Outdated Fields
Get contextual knowledge about the ticket through the 'Summary', 'Description', and 'Comments' fields.
Identify any missing or outdated fields and form recommendations for updates.
Return a list with the changes you made with the addition of recommendations and nothing else.
Each list item should be an object in the following format:
"field": "<name of the missing or outdated field>",
"recommendation": "<recommendation on how to update the field and why>"

If you do not have to update fields, return "Everything is up to date"."""

update_prompt = PromptTemplate(template= update_temp, input_variables= [])
update_prompt.save("prompts/update/updatePrompt_V1.12.2.json")

# Summary Length Prompt (Best Practice)

- 0-shot
- ??

In [4]:
sumLen_temp = """Best Practice: Summary Length
Your Task is to check if the ticket summary has exactly between {min} and {max} characters.
If the number of characters is exactly in this range, return the original summary.
Otherwise, recommend a new summary based on the information of the description field for the ticket.
Please return only the summary and nothing else."""

sumLen_prompt = PromptTemplate(template= sumLen_temp, input_variables= ["min", "max"])

sumLen_prompt.save("prompts/summary/summaryLengthPrompt_V2.2.0.json")

# Description Length Prompt (Best Practice)

- 0-shot
- ??

In [2]:
descLen_temp = """Best Practice: Description Length
Your Task is to check if the ticket description has approximately between {min} and {max} characters.
Ignore source code snippets or images in the description.
If the number of characters is in this range, return the original description.
Otherwise, recommend a new description.
Keep this description more technical and less requesting or asking for help.
Return only the description string and nothing else."""

descLen_prompt = PromptTemplate(template= descLen_temp, input_variables= ["min", "max"])

descLen_prompt.save("prompts/description/descriptionLengthPrompt_V2.7.0.json")

# Toxic Speech Prompt (Best Practice)

- 0-shot
- ??

In [19]:
toxicSpeech_temp = """Best Practice: Toxic Speech Detection
Your task is to check the comments section for toxic speech.
If you detect toxic speech, return a list with the comment, the author, and the datetime.
Each list item should be an object in the following format:
"comment": "<content of the original comment>",
"author": "<name of the original comment>",
"datetime": "<datetime of the original comment>",
"recommendation": "<recommendation on how to update the comment and why>"

If you do not detect toxic speech, return "no toxic speech detected"."""

toxicSpeech_prompt = PromptTemplate(template= toxicSpeech_temp, input_variables= [])

toxicSpeech_prompt.save("prompts/toxicSpeech/toxicSpeechPrompt_V1.2.4.json")

# Internationalization Prompt (Best Practice)

- 0-shot
- ??

In [13]:
internationalization_temp = """Best Practice: Internationalization
Bug reports not written in English are often closed immediately or ignored by developers.
Your task is to check if the ticket is written in English.
If some fields of the ticket are not written in English, provide a translation.
Return only the translated fields in the following structure and nothing else.
"<field>": "<English translation of the original field>"
"""

internationalization_prompt = PromptTemplate(template= internationalization_temp, input_variables= [])

internationalization_prompt.save("prompts/internationalization/internationalizationPrompt_V1.2.0.json")

# Bug Report Structure

- 0-shot
- few-shot (to show tell the model how to restructure the avaiable data into the given structure)

In [4]:
examples = [
    {
        "input": """{{'Summary':'Sourcetree crashes',
'Description':'Source tree crashes everytime i open , I have uninstalled and installed many times',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'Git',
'CreatedDate':'2021-09-13T05:48:08.000+0000',
'ResolvedDate':'2021-11-18T15:12:59.000+0000',
'Status':'Closed',
'Priority':'Low',
'Creator':'John Doe',
'Reporter':'John Doe',
'Resolution':'Cannot Reproduce',
'IssueLinks':null,
'Labels':null,
'VersionsFixed':null,
'Assignee':null,
'TimeSpent':null,
'Comments':[
{{
'Author':'Max Mustermann',
'Created':'2021-11-18 15:12:59.861000+00:00',
'Comment':'Issue was fixed in latest Sourcetree versions. Please use Sourcetree 3.4.5 > '
}}
]
}}""",
        "output": """
1. Overview
Source tree crashes everytime i open , I have uninstalled and installed many times
2. Steps to reproduce
!not provided
3. Actual result
Source tree crashes
4. Expected result
Source tree does not crash
5. Stack Traces
!not provided
6. Build date and hardware
!not provided
7. Additional information
!not provided
""" 
    },
    {
"input": """
{{
'Summary':'Opening and closing Sourcetree causes Pageant to crash\/close',
'Description':'I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready.\r\n\r\nDidn't quite notice it at first, so not sure which version it started at, but Sourcetree now causes Pageant to crash\/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason.\r\n\r\nWhenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. Sourcetree shouldn't close or cause any other applications to crash when it opens or when it closes.\r\n\r\nIt happens on all 3 computers of mine. All running latest Windows, latest Pageant, and latest Sourcetree.',
'VersionsAffected':'3.4.4',
'IssueType':'Bug',
'Project':'Sourcetree for Windows',
'Components':'General',
'CreatedDate':'2021-04-26T08:26:07.000+0000',
'ResolvedDate':'2021-04-27T06:38:20.000+0000',
'Status':'Closed',
'Priority':'Low',
'Creator':'Jane Doe',
'Reporter':'Jane Doe',
'Resolution':'Duplicate',
'IssueLinks':null,
'Labels':'verified',
'VersionsFixed':null,
'Assignee':null,
'TimeSpent':null,
'Comments':[
{{
'Author':'Jane Doe',
'Created':'2021-04-26 08:41:02.391000+00:00',
'Comment':'I have tried to disable the \"Automatically start SSH agent when Sourcetree opens\" option, as I thought maybe that was what was messing with things, but that didn't change anything. Pageant still disappears from my systray when Sourcetree opens or closes.'
}},
{{
'Author':'Jane Doe',
'Created':'2021-04-26 08:42:41.095000+00:00',
'Comment':'Just noticed Sourcetree even causes Pageant to crash\/close when I close the Options dialog... \ud83d\ude44'
}},
{{
'Author':'Jonathan Smith',
'Created':'2021-04-26 10:07:20.880000+00:00',
'Comment':'Known issue, duplicate. Please use ticket\u00a0SRCTREEWIN-13475\u00a0for suggestions and tracking updates.'
}}
]
}}""",
"output": """
1. Overview
I have installed putty and pageant outside of Sourcetree, and I have Pageant startup during OS boot so my key-file is always ready. Didn't quite notice it at first, but Sourcetree now causes Pageant to crash/close when it opens or closes, causing e.g. git commands and such from command line to suddenly start to failing for no apparent reason. Whenever I open or close Sourcetree now, I have to reopen Pageant and reload my key, including entering a long passkey. Super annoying. It happens on all 3 computers of mine.             
2. Steps to reproduce
- Install putty and pageant outside of Sourcetree
- Set Pageant to startup during OS boot
- pen or close Sourcetree
3. Actual result
Sourcetree causes Pageant to crash\/close when it opens or closes.
4. Expected result
Sourcetree should not close or cause any other applications to crash when it opens or when it closes.
5. Stack Traces
!not provided
6. Build date and hardware
All running latest Windows, latest Pageant, and latest Sourcetree.
Not sure which version it started.
7. Additional information
- I have tried to disable the "Automatically start SSH agent when Sourcetree opens" option, as I thought maybe that was what was messing with things, but that didn't change anything. Pageant still disappears from my systray when Sourcetree opens or closes.
- Just noticed Sourcetree even causes Pageant to crash/close when I close the Options dialog.
""" 
    }
]

In [5]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"], 
    template="""### Context ###
A bug report is a document that describes an issue with a product or service and why it needs fixing.
The description field is significant for the bug report, as it describes the issue in detail.
The description should contain the following structure:
1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
2. Steps to reproduce: A list of steps to reproduce the error.
3. Actual result: The result that is currently produced.
4. Expected result: The result that is expected.
5. Stack Traces: A list of relevant stack traces for the bug report.
6. Build date and hardware: The build date and hardware, when and where the bug occurred.
7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Rearrange the description field of the bug report to match the structure described above.
If some information is missing, give suggestions and add them to the description field, if possible.
Return the new description field and nothing else.

The bug report is provided in JSON format.
Bug Report: {input} \n {output}"""
)

# print(example_prompt.format(**examples[0]))

In [6]:
bugReportStructure_fs_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= """### Context ###
A bug report is a document that describes an issue with a product or service and why it needs fixing.
The description field is significant for the bug report, as it describes the issue in detail.
The description should contain the following structure:
1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
2. Steps to reproduce: A list of steps to reproduce the error.
3. Actual result: The result that is currently produced.
4. Expected result: The result that is expected.
5. Stack Traces: A list of relevant stack traces for the bug report.
6. Build date and hardware: The build date and hardware, when and where the bug occurred.
7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Rearrange the description field of the bug report to match the structure described above.
If some information is missing, give suggestions and add them to the description field, if possible.
Return the new description field and nothing else.

The bug report is provided in JSON format.
Bug Report: {bug_report}""",
    input_variables=["bug_report"]
)

bugReportStructure_fs_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_V2.1.0.json")

In [11]:
# bugReportStructure_temp = """### Context ###
# A bug report is a document that describes an issue with a product or service and why it needs fixing.
# The description field is significant for the bug report, as it describes the issue in detail.
# The description should contain the following structure:
# 1. Overview: This is more detailed than the summary field of the bug report, with a description of the context when the error occurred.
# 2. Steps to reproduce: A list of steps to reproduce the error.
# 3. Actual result: The result that is currently produced.
# 4. Expected result: The result that is expected.
# 5. Stack Traces: A list of relevant stack traces for the bug report.
# 6. Build date and hardware: The build date and hardware, when and where the bug occurred.
# 7. Additional information: Additional information relevant to the bug report. That can be screenshots, videos, or other files.

# ### Instruction ###
# Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
# Rearrange the description field of the bug report to match the structure described above.
# If some information is missing, give suggestions and add them to the description field, if possible.
# Return the new description field and nothing else.

# The bug report is provided in JSON format.
# Bug Report: {bug_report}
# """

# bugReportStructure_prompt = PromptTemplate(
#     template=bugReportStructure_temp, 
#     input_variables=["bug_report"])

# bugReportStructure_prompt.save("prompts/bugReportStructure/bugReportStructurePrompt_V1.1.1.json")